# CS 484 :: Data Mining :: George Mason University :: Spring 2023


# Homework 3: Clustering&Association Rule Mining

- **100 points [9% of your final grade]**
- **Due Sunday, April 16 by 11:59pm**

- *Goals of this homework:* (1) implement your K-means model; and (2) implement the association rule mining process with the Apriori algorithm.

- *Submission instructions:* for this homework, you only need to submit to Blackboard. Please name your submission **FirstName_Lastname_hw3.ipynb**, so for example, my submission would be something like **Ziwei_Zhu_hw3.ipynb**. Your notebook should be fully executed so that we can see all outputs. 

## Part 1: Clustering (50 points)

In this part, you will implement your own K-means algorithm to conduct clustering on handwritten digit images. In this homework, we will still use the handwritten digit image dataset we have already used in previous homework. However, since clustering is unsupervised learning, which means we do not use the label information anymore. So, here, we will only use the testing data stored in the "test.txt" file.

First, let's load the data by excuting the following code:

In [16]:
import numpy as np

test = np.loadtxt("test.txt", delimiter=',')
test_features = test[:, 1:]
test_labels = test[:, 0]
print('array of testing feature matrix: shape ' + str(np.shape(test_features)))
print('array of testing label matrix: shape ' + str(np.shape(test_labels)))

array of testing feature matrix: shape (10000, 784)
array of testing label matrix: shape (10000,)


Now, it's time for you to implement your own K-means algorithm. First, please write your code to build your K-means model using the image data with **K = 10**, and **Euclidean distance**.

**Note: You should implement the algorithm by yourself. You are NOT allowed to use Machine Learning libraries like Sklearn**

**Note: you need to decide when to stop the model training process.**

In [17]:

# First randomly select k data points for initializing centroids.
def Centroids(d, k):
    
    # Amount of data points
    num = d.shape[0]
    
    # Choosing k indexes randomly
    indices = np.random.choice(num, k, replace=False)
    
    # Getting centroids
    centroids = d[indices]
    
    return (centroids)

# Euclidean distance method
def Euclidean(a, b):
    
    # Getting the squared distances
    difference = ((a - b) ** 2)
    
    # Summing the squared differences
    sum = np.sum(difference, axis=1)
    
    # Square root for euclidean distance
    distance = np.sqrt(sum)
    
    return (distance)


# Method for going through each centroid and distance
def Assign(d, c):
    
    # Storing distances
    distances = np.zeros((d.shape[0], c.shape[0]))
    
    # Getting distances
    for i in range(len(c)):
        
        distances[:, i] = Euclidean(d, c[i])
    
    # Assigning
    assignments = np.argmin(distances, axis=1)
    
    return (assignments)

# Method to update centroid
def Update(d, a, k):
    
    # Storing centroids
    new = np.zeros((k, d.shape[1]))
    
    # Getting mean
    for i in range(k):
        
        ap = d[a == i]
        
        new[i] = np.mean(ap)
    
    return (new)

# K-means using helpers
def kmeans(d, k, max=100, min=0.01):
    
    # Initializing centroids
    centroids = Centroids(d, k)
    
    # For comparing
    previous = np.zeros(d.shape[0])
    
    # Updating centroids and assignments
    for _ in range(max):
        # Assigning to clusters
        assignments = Assign(d, centroids)
        
        # Stop when there is convergence
        changes = np.sum(assignments != previous)
        
        if ((changes / d.shape[0]) < min):
            
            break
        
        # Updating centroids from current assignments
        centroids = Update(d, assignments, k)
        
        # Updating previous assignments for the next comparison
        previous = assignments.copy()
    
    return (centroids, assignments)


Next, you need to calculate the square root of Sum of Squared Error (SSE) of each cluster generated by your K-means algorithm. Then, print out the averaged SSE of your algorithm.

Then, please have a look on https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_completeness_v_measure.html#sklearn.metrics.homogeneity_completeness_v_measure, and use this function to print out the homogeneity, completeness, and v-measure of your K-means model.

## Part 2: Association Rule Mining (50 points)

In this part, you are going to examine movies using our understanding of association rules. For this part, you need to implement the apriori algorithm, and apply it to a movie rating dataset to find association rules of user-rate-movie behaviors. First, run the next cell to load the dataset we are going to use.

In [18]:
user_movie_data = np.loadtxt("movie_rated.txt", delimiter=',')
print('array of user-movie matrix: shape ' + str(np.shape(user_movie_data)))

array of user-movie matrix: shape (11743, 2)


In this dataset, there are two columns: the first column is the integer ids of users, and the second column is the integer ids of movies. Each row denotes that the user of given user id rated the movie of the given movie id. We are going to treat each user as a transaction, so you will need to collect all the movies that have been rated by a single user as a transaction. 

Now, you need to implement the apriori algorithm and apply it to this dataset to find association rules of user rating behaviors with **minimum support of 0.2** and **minimum confidence of 0.8**. We know there are many existing implementations of apriori online (check github for some good starting points). You are welcome to read existing codebases and let that inform your approach. 

**Note: Do not copy-paste any existing code.**

**Note: We want your code to have sufficient comments to explain your steps, to show us that you really know what you are doing.**

**Note: You should add print statements to print out the intermediate steps of your method -- e.g., the size of the candidate set at each step of the method, the size of the filtered set, and any other important information you think will highlight the method.**

In [19]:
# Method for getting the initial transaction candidates
def Initialize(transactions):
    
    candidates = []    
    
    for transaction in transactions:    
    
        for item in transaction:    
    
            if [item] not in candidates:    
    
                candidates.append([item])
    
    return (candidates)

# Method for getting the supports 
def Support(count, num):
    
    supports = {}
    
    for key, value in count.items():
    
        supports[key] = (value / num)
    
    return (supports)

# Method for filtering the candidates 
def Filter(transactions, candidates, min_support):
    
    count = {}
    
    for transaction in transactions:
    
        for candidate in candidates:
    
            if set(candidate).issubset(transaction):
    
                tuple_candidate = tuple(candidate)
    
                count[tuple_candidate] = (count.get(tuple_candidate, 0) + 1)
                
    supports = Support(count, float(len(transactions)))
    
    filtered = []
    
    for key, support in supports.items():
    
        if (support >= min_support):
            
            filtered.append(list(key))
    
    return (filtered)

# Method for making new candidate sets
def New(filtered, k):
    
    new_candidates = []
    
    for i in range(len(filtered)):
    
        for j in range(i+1, len(filtered)):
    
            L1 = sorted(filtered[i])[:k-2]
    
            L2 = sorted(filtered[j])[:k-2]
    
            if (L1 == L2):
                
                new_candidates.append(sorted(set(filtered[i]) | set(filtered[j])))
    
    return (new_candidates)

# The apriori algorithm using helpers
def Apriori(transactions, min=0.2):
    
    candidates = Initialize(transactions)
    
    all_frequent_itemsets = []
    
    k = 2
    
    while candidates:
    
        filtered = Filter(transactions, candidates, min)
    
        # For all candidates
        print(f"Size of the candidate set (k={k-1}): {len(candidates)}")
    
        # For all filtered
        print(f"Size of the filtered set (k={k-1}): {len(filtered)}")
    
        if not filtered:
    
            break
    
        all_frequent_itemsets.extend(filtered)
    
        candidates = New(filtered, k)
    
        k += 1
    
    return (all_frequent_itemsets)

transactions = []

unique_users = np.unique(user_movie_data[:, 0])

for user in unique_users:

    user_movies = user_movie_data[user_movie_data[:, 0] == user][:, 1]

    transactions.append(list(user_movies))

# Using the apriori algorithm
frequent_itemsets = Apriori(transactions, min=0.2)

frequent_itemsets[:5]



Size of the candidate set (k=1): 408
Size of the filtered set (k=1): 21
Size of the candidate set (k=2): 210
Size of the filtered set (k=2): 36
Size of the candidate set (k=3): 105
Size of the filtered set (k=3): 12
Size of the candidate set (k=4): 7
Size of the filtered set (k=4): 0


[[480.0], [1221.0], [1270.0], [1265.0], [1197.0]]